In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ohmeow-blurr -q
!pip install bert-score -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import pandas as pd
from fastai.text.all import *
from transformers import (
    BartForConditionalGeneration,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
from blurr.text.data.all import *
from blurr.text.modeling.all import *

In [5]:
#Get data
df = pd.read_csv('./drive/MyDrive/output.csv', sep=',')
df = df.dropna().reset_index()

#Select part of data we want to keep
df = df[['text','summary']]

#Clean text
df['text'] = df['text'].apply(lambda x: x.replace('\n',''))
df['summary'] = df['summary'].apply(lambda x: x.replace('\n',''))

#Select only part of it (makes testing faster)
writeups = df.head()
writeups.head()

,text,summary
0,"When you visit the website, you get redirected to `/?file=wc.php`. This might indicate that you can include files from the server, such as `/?file=/etc/passwd`. You can see in this file that there's a user called `ctf`, but that's not useful yet.Moving on, you can find out that there's a `robots.txt` file at `?file=robots.txt`.In it there is the `/?file=checkpass.php` file disallowed.Visiting that URL, you get redirected back to `/wc.php`. However, it maybe that there's some code in `checkpass.php` that might be important. If you request it in `python` using `requests`, you can see that th...","Visit website and notice the `/?file=wc.php` redirect, indicating possible file inclusion.Explore `?file=/etc/passwd` and find `ctf` user.Check `?file=robots.txt` and find `/?file=checkpass.php` disallowed.Use `php://filter` to view the source of `checkpass.php`, find hardcoded password.Check the source of `wc.php`, set the cookie with the password, and notice remote code execution.Use a reverse shell payload to connect to the server.Navigate through the file system to find the `flag.txt` file.Find the `ctf` user's password hash in the `README` file and crack it.Switch to the `ctf` user us..."
1,"Description: ""You can steal a car if you steal its key.""First, in the website source, you can see that there's a route to login (`/login`) and a route to get the names of the admins (`/adminNames`). So, when you try to visit `/login`, you see a form. Post that and check the response on the Network Tab in your browser. You can see there's a header called `token` which stores a JWT. You can decode the token (maybe on `jwt.io` to see that the payload is the following (I typed username: asd, password: asd):`{ ""username"": ""nfq"", ""password"": ""nfq"", ""admin"": ""snyfr"", ""iat"": 1593506966 }`We can se...","Examine the website source to find routes `/login` and `/adminNames`.Fill out the login form and inspect the Network Tab in your browser to find the token header containing a JWT.Decode the JWT to find the payload, then apply the `rot13` cipher to the `admin` value.Visit the `/adminNames` route to download a file containing a GitHub URL, and find the list of admin usernames in the repository.Modify the JWT payload with a rot13 encrypted admin username and change the admin value to the rot13 encrypted string for `true`.Find the JWT secret by trying the `/getFile?file=../.env` route to downl..."
2,"Description: ""This is my file library. I don't have a lot of files, but I hope you like the ones I have!""This exploit is owing to the `qs` package used by `express` to parse `req.query`. You can send an array instead of a string through the GET params in the following manner:[GET request with an array as a parameter]This makes the `req.names` in the backend an array: `['hello', 'world']`.[JavaScript code of a simple Express app with a /getFile route and other routes]The `/getFile` route is meant to get you a file from the server. But there are certain restrictions. First, you can't have ` ...","Analyze the JavaScript code of the Express app and understand the restrictions on the `file` parameter.Inspect the `a.cpp` file and notice the `system(""cat flag.txt"")` command, indicating the presence of `flag.txt` in the same directory.Craft a payload to bypass the restrictions and use `path.resolve()` to access `flag.txt`. Payload: `/getFile?file[]=a&file[]=b&file[]=c&file[]=d&file[]=/../flag.txt&file[]=.&file[]=js`Send the payload to retrieve the flag."
3,"Description: ""People who get violent get that way because they can't communicate.""When you google the challenge description, you find out that the quote is from Mr. Robot. This indicates that the user might want to check out the `robots.txt` for the website.When you open the website, it serves the `index.html` file, which has content written about `Brobot`, again trying to put across `robots.txt`.When you visit `robots.txt` and visit the disallowed route, you get the flag!","Goo

In [8]:
#Import the pretrained model
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

#Create mini-batch and define parameters
hf_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
    task='summarization',
    text_gen_kwargs=
 {'max_length': 248,'min_length': 56,'do_sample': False, 'early_stopping': True, 'num_beams': 4, 'temperature': 1.0, 
  'top_k': 50, 'top_p': 1.0, 'repetition_penalty': 1.0, 'bad_words_ids': None, 'bos_token_id': 0, 'pad_token_id': 1,
 'eos_token_id': 2, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1, 'decoder_start_token_id': 2, 'use_cache': True, 'num_beam_groups': 1,
 'diversity_penalty': 0.0, 'output_attentions': False, 'output_hidden_states': False, 'output_scores': False,
 'return_dict_in_generate': False, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2, 'remove_invalid_values': False})


#Prepare data for training
blocks = (HF_Seq2SeqBlock(before_batch_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('text'), get_y=ColReader('summary'), splitter=RandomSplitter())
dls = dblock.dataloaders(writeups, batch_size = 2)

NameError: ignored

In [ ]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}

#Model
model = HF_BaseModelWrapper(hf_model)
learn_cbs = [HF_BaseModelCallback]
fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

#Specify training
learn = Learner(dls, model,
                opt_func=ranger,loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()

#Create optimizer with default hyper-parameters
learn.create_opt() 
learn.freeze()

#Training
learn.fit_one_cycle(3, lr_max=3e-5, cbs=fit_cbs)

In [ ]:
outputs = learn.blurr_generate(text_to_generate, early_stopping=False, num_return_sequences=1)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')